### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [5]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

### read in data

In [57]:
df = pd.read_csv("data/Discovery StatisticsPrintDownload.csv")

In [58]:
df.columns = df.columns.str.lower()

In [59]:
df["date"] = pd.to_datetime(df["date"])
df["small_pha"] = df["pha"] - df["pha-km"]

In [60]:
df = df.sort_values("date", ascending=True)
df["new_pha_total"] = df["pha"] - df["pha"].shift(periods=1)
df["new_pha_km"] = df["pha-km"] - df["pha-km"].shift(periods=1)
df["new_pha_small"] = df["small_pha"] - df["small_pha"].shift(periods=1)

In [61]:
df_slim = df[df["date"] > "2010-01-01"][
    [
        "date",
        "pha-km",
        "small_pha",
        "pha",
        "new_pha_km",
        "new_pha_small",
        "new_pha_total",
    ]
]

In [11]:
#dw.add_data(chart_id="wPjiz", data=df_slim)
# chart_id='wPjiz'

<Response [204]>

In [12]:
new_phas = pd.melt(
    df_slim, id_vars=["date"], value_vars=["new_pha_km", "new_pha_small"]
)

In [13]:
alt.Chart(new_phas).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value"), color="variable"
)

alt.Chart(...)

In [14]:
alt.Chart(new_phas).mark_circle(opacity=0.8, stroke="white", strokeWidth=1).encode(
    alt.X("date:O", axis=alt.Axis(labelAngle=0)),
    alt.Y("variable:N"),
    alt.Size("value"),
    alt.Color("variable:N", legend=None),
).properties(width=650, height=320)

alt.Chart(...)

In [15]:
import datetime as dt

In [63]:
df_slim["year"] = df_slim["date"].dt.year

In [64]:
## do a top and bottom panel
annual_phas = (
    df_slim.groupby("year")
    .agg(
        annual_pha_km=pd.NamedAgg(column="new_pha_km", aggfunc=sum),
        annual_pha=pd.NamedAgg(column="new_pha_small", aggfunc=sum),
        annual_pha_total=pd.NamedAgg(column="new_pha_total", aggfunc=sum),
    )
    .reset_index()
)

annual_phas_long = pd.melt(
    annual_phas,
    id_vars=["year"],
    value_vars=["annual_pha_km", "annual_pha", "annual_pha_total"],
)

In [46]:
annual_phas_long["y"] = 1

In [53]:
bubbles = (
    alt.Chart(annual_phas_long[annual_phas_long["variable"] == "annual_pha_km"])
    .mark_circle(opacity=0.8, stroke="white", strokeWidth=1, fill="#E37E2D")
    .encode(
        alt.X("year:O", axis=alt.Axis(title="", tickSize=0)),
        alt.Y(
            "y",
            scale=alt.Scale(zero=False),
            axis=alt.Axis(title="PHAs > 1 km", tickCount=0),
        ),
        alt.Size("value", legend=alt.Legend(title="Count")),
    )
    .properties(width=350, height=100)
)

In [54]:
line = (
    alt.Chart(annual_phas_long[annual_phas_long["variable"] == "annual_pha"])
    .mark_line(
        point=alt.OverlayMarkDef(filled=True, fill="#00D4D8", color="#00D4D8"),
        color="#00D4D8",
    )
    .encode(
        alt.X("year:O", axis=alt.Axis(labelAngle=0)),
        alt.Y("value", axis=alt.Axis(title="Annual PHAs < 1 km")),
    )
    .properties(width=350, height=320)
)

In [55]:
alt.vconcat(
    bubbles, line, data=annual_phas_long, title="Potentially Hazardous Asteroids"
)

alt.VConcatChart(...)

In [22]:
alt.Chart(annual_phas_long).mark_line(
    point=alt.OverlayMarkDef(filled=True, fill="#00D4D8", color="#00D4D8"),
    color="#00D4D8",
).encode(
    alt.X("year:O", axis=alt.Axis(labelAngle=0)),
    alt.Y("value"),
    alt.Color("variable:N"),
).properties(
    width=350, height=320
)

alt.Chart(...)

In [39]:
alt.Chart(annual_phas[annual_phas["variable"] == "annual_pha_km"]).mark_bar().encode(
    x="year:O",
    y="value",
    color="variable",
    order=alt.Order("variable", sort="ascending"),
)

alt.Chart(...)

In [34]:
df_long = pd.melt(df_slim, id_vars=["date"], value_vars=["new_pha_km", "new_pha_small"])

In [38]:
alt.Chart(df_long[df_long["variable"] == "new_pha_km"]).mark_bar().encode(
    x="date:O",
    y="value",
    color="variable",
    order=alt.Order("variable", sort="descending"),
)

alt.Chart(...)

In [23]:
annual_phas_dw = (
    df_slim.groupby("year")
    .agg(
        annual_pha_km=pd.NamedAgg(column="new_pha_km", aggfunc=sum),
        annual_pha=pd.NamedAgg(column="new_pha_small", aggfunc=sum),
        annual_pha_total=pd.NamedAgg(column="new_pha_total", aggfunc=sum),
    )
    .reset_index()
)

In [68]:
# dw.create_chart(
#    "Annual Potentially Hazardous Asteroids", chart_type="d3-lines", data=annual_phas
# )
chart_id = "igiM3"
dw.add_data(chart_id=chart_id, data=annual_phas[annual_phas["year"] != "2010"])

<Response [204]>